In [2]:
## Set notebook to auto reload updated modules
%load_ext autoreload
%autoreload 2

In [11]:
from cflarepy.libs import core_utils, setup
from cflarepy.libs.settings import CLOUDFLARE_SETTINGS

In [4]:
import pandas as pd
from cloudflare import Cloudflare

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
def get_zones(cf_client: Cloudflare):
    zones = []

    display("Getting Cloudflare zones")
    try:
        for zone in cf_client.zones.list():
            zones.append(zone)
    except Exception as exc:
        msg = f"({type(exc)}) Error getting Cloudflare zones. Details: {exc}"
        display(f"[ERROR] {msg}")
        
        raise exc
        
    return zones

In [ ]:
import os
display(os.environ.get("CF_API_KEY"))

In [ ]:
os.environ.items()

In [ ]:
CLOUDFLARE_SETTINGS.as_dict()

In [ ]:
api_token = CLOUDFLARE_SETTINGS.get("CF_API_KEY")
if not api_token:
    raise ValueError("Missing Cloudflare API token. Set environment variable CF_API_KEY=<API_KEY>")

display(api_token)


In [ ]:
cf_client =  Cloudflare(api_token=settings.CLOUDFLARE_SETTINGS.get("CF_API_KEY"))

In [ ]:
zones = get_zones(cf_client=cf_client)
zones